In [1]:
pip install unicode

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk import tokenize
import nltk
from gensim.models import KeyedVectors
from string import punctuation
import unidecode
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sthem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Preparing Data

In [3]:
colnames=['output','text']
df = pd.read_csv('all-data.csv', names=colnames, encoding = "ISO-8859-1", header=None)

In [4]:
all_sentences = [text for text in df.text]
words = ' '.join(all_sentences)

In [5]:
stop_words = nltk.corpus.stopwords.words('english')
token_space = tokenize.WhitespaceTokenizer()
token_punct = tokenize.WordPunctTokenizer()

In [6]:
puncts = list()
for punct in punctuation:
    puncts.append(punct)
punct_and_stopwords = puncts + stop_words

In [7]:
sentences_after_stopwords = list()

for sentence in df.text:
    new_sentence = list()
    words_from_sentence = token_space.tokenize(sentence)
    for word in words_from_sentence:
        if word not in stop_words:
            new_sentence.append(word)
    sentences_after_stopwords.append(" ".join(new_sentence))

In [8]:
sentences_after_stopwords_puncts = list()

for sentence in sentences_after_stopwords:
    for punct_to_change in punctuation:
        sentence = sentence.replace(punct_to_change,"")
    sentences_after_stopwords_puncts.append(sentence)

In [9]:
sentences_after_stopwords_puncts_lower = list()

for sentence in sentences_after_stopwords_puncts:
    sentence = sentence.lower()
    sentences_after_stopwords_puncts_lower.append(sentence)

In [10]:
# sentences_after_stopwords_puncts = list()

# for sentence in sentences_after_stopwords:
#     new_sentence = list()
#     words_from_sentence = token_punct.tokenize(sentence)
#     # print(words_from_sentence)
#     for word in words_from_sentence:
#         if word not in puncts:
#             new_sentence.append(word)
#     # print(new_sentence)
#     sentences_after_stopwords_puncts.append(" ".join(new_sentence))

In [11]:
stop_words_no_accent = list()
for word in stop_words:
    for punct_to_change in punctuation:
        word = word.replace(punct_to_change,"")
    stop_words_no_accent.append(word)

In [12]:
sentences_after_stopwords_puncts_lower_stopwords = list()

for sentence in sentences_after_stopwords_puncts_lower:
    new_sentence = list()
    words_from_sentence = token_space.tokenize(sentence)
    for word in words_from_sentence:
        if word not in stop_words_no_accent:
            new_sentence.append(word)
    sentences_after_stopwords_puncts_lower_stopwords.append(" ".join(new_sentence))

In [13]:
sentences_after_stopwords_puncts_lower_stopwords_number = list()

for sentence in sentences_after_stopwords_puncts_lower_stopwords:
    new_sentence = list()
    words_from_sentence = token_space.tokenize(sentence)
    for word in words_from_sentence:
        if not word.isnumeric():
            new_sentence.append(word)
        else:
            new_sentence.append("0")
    sentences_after_stopwords_puncts_lower_stopwords_number.append(" ".join(new_sentence))

Resampling

In [14]:
clean_df = pd.DataFrame({'sentences':sentences_after_stopwords_puncts_lower_stopwords_number,'output':df.output})

In [20]:
pip install -U imbalanced-learn

  Using cached imbalanced_learn-0.9.0-py3-none-any.whl (199 kB)
  Using cached scikit_learn-1.0.2-cp39-cp39-win_amd64.whl (7.2 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Acesso negado: 'c:\\programdata\\anaconda3\\lib\\site-packages\\scikit_learn-0.24.2.dist-info\\COPYING'
Consider using the `--user` option or check the permissions.



In [21]:
from imblearn.over_sampling import SMOTE
import numpy as np
smote = SMOTE(random_state=100)

In [22]:
tfidf = TfidfVectorizer(lowercase=False,max_features=600)
vector_tfidf = tfidf.fit_transform(sentences_after_stopwords_puncts_lower_stopwords_number)

In [23]:
X_to_be_transformed = vector_tfidf
Y_to_be_transformed = clean_df.output
X_resampled, Y_resampled = smote.fit_resample(X_to_be_transformed,Y_to_be_transformed)

Y_resampled.value_counts()

neutral     2879
negative    2879
positive    2879
Name: output, dtype: int64

In [24]:
X_train,X_test,Y_train,Y_test = train_test_split(X_resampled,Y_resampled,random_state = 100,test_size=0.1)

In [25]:
logistic_regression = LogisticRegression(solver = 'lbfgs')

In [ ]:
logistic_regression.fit(X_train,Y_train)
logistic_regression.score(X_test,Y_test)

LogisticRegression Model

In [ ]:
logistic_regression = LogisticRegression(solver = 'lbfgs')

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(vector_tfidf,df.output,random_state = 100,test_size=0.1)

In [ ]:
# Score using all data without any transformation
logistic_regression.fit(X_train,Y_train)
logistic_regression.score(X_test,Y_test)

In [ ]:
Y_predicted = list(logistic_regression.predict(X_test))
Y_test_list = list(Y_test)

cont = 0
for i in range(len(Y_test)):
    if Y_test_list[i] == Y_predicted[i]:
        cont += 1
    
print("Score:",cont/len(Y_test))

In [ ]:
# all_words = ' '.join(sentences_after_stopwords_puncts_lower_stopwords_number)
# cloud_of_words = WordCloud().generate(all_words)

In [ ]:
# plt.figure(figsize=(10,8))
# plt.imshow(cloud_of_words, interpolation='bilinear')
# plt.show()

In [ ]:
print(logistic_regression.predict(X_test[0]))
print(logistic_regression.predict_proba(X_test[0]))
print(Y_test.iloc[0])

In [ ]:
logistic_regression.predict_proba(X_test)

In [ ]:
logistic_regression.classes_

LogisticRegression Testing

In [ ]:
prob_class = list(logistic_regression.predict_proba(X_test))

cont_80 = 0
cont = 0
new_column = []

for line in prob_class:
    if line.max() >= 0.800:
        position = list(line).index(line.max())
        classification = logistic_regression.classes_[position]
        cont_80 += 1
        new_column.append(classification)
    else:
        new_column.append('Under 0.8')
        

In [ ]:
Y_predicted = list(logistic_regression.predict(X_test))
Y_test_list = list(Y_test)

cont = 0
for i in range(len(Y_test)):
    if Y_test_list[i] == new_column[i]:
        cont += 1
    
print("Qnt:",cont_80)
print("Score:",cont/cont_80)

LogisticRegression Analysing External Phrases

In [ ]:
sentence_to_transform = ["The market is bad"]


In [ ]:
sentences_after_stopwords = list()

for sentence in sentence_to_transform:
    new_sentence = list()
    words_from_sentence = token_space.tokenize(sentence)
    for word in words_from_sentence:
        if word not in stop_words:
            new_sentence.append(word)
    sentences_after_stopwords.append(" ".join(new_sentence))

In [ ]:
sentences_after_stopwords_puncts = list()

for sentence in sentences_after_stopwords:
    for punct_to_change in punctuation:
        sentence = sentence.replace(punct_to_change,"")
    sentences_after_stopwords_puncts.append(sentence)

In [ ]:
sentences_after_stopwords_puncts_lower = list()

for sentence in sentences_after_stopwords_puncts:
    sentence = sentence.lower()
    sentences_after_stopwords_puncts_lower.append(sentence)

In [ ]:
sentences_after_stopwords_puncts_lower_stopwords = list()

for sentence in sentences_after_stopwords_puncts_lower:
    new_sentence = list()
    words_from_sentence = token_space.tokenize(sentence)
    for word in words_from_sentence:
        if word not in stop_words_no_accent:
            new_sentence.append(word)
    sentences_after_stopwords_puncts_lower_stopwords.append(" ".join(new_sentence))

In [ ]:
sentences_after_stopwords_puncts_lower_stopwords_number = list()

for sentence in sentences_after_stopwords_puncts_lower_stopwords:
    new_sentence = list()
    words_from_sentence = token_space.tokenize(sentence)
    for word in words_from_sentence:
        if not word.isnumeric():
            new_sentence.append(word)
        else:
            new_sentence.append("0")
    sentences_after_stopwords_puncts_lower_stopwords_number.append(" ".join(new_sentence))

In [ ]:
print(sentences_after_stopwords_puncts_lower_stopwords_number)
vector_tfidf_sentence = tfidf.transform(sentences_after_stopwords_puncts_lower_stopwords_number)

In [ ]:
print(logistic_regression.predict(vector_tfidf_sentence))
print(logistic_regression.predict_proba(vector_tfidf_sentence))

Creating a Function to transform the sentence

In [ ]:
def transformSentence(list_of_sentences):
    
    sentences_after_stopwords = list()
    
    for sentence in list_of_sentences:
        new_sentence = list()
        words_from_sentence = token_space.tokenize(sentence)
        for word in words_from_sentence:
            if word not in stop_words:
                new_sentence.append(word)
        sentences_after_stopwords.append(" ".join(new_sentence))

    sentences_after_stopwords_puncts = list()

    for sentence in sentences_after_stopwords:
        for punct_to_change in punctuation:
            sentence = sentence.replace(punct_to_change,"")
        sentences_after_stopwords_puncts.append(sentence)

    sentences_after_stopwords_puncts_lower = list()

    for sentence in sentences_after_stopwords_puncts:
        sentence = sentence.lower()
        sentences_after_stopwords_puncts_lower.append(sentence)

    stop_words_no_accent = list()

    for word in stop_words:
        for punct_to_change in punctuation:
            word = word.replace(punct_to_change,"")
        stop_words_no_accent.append(word)
    
    sentences_after_stopwords_puncts_lower_stopwords = list()

    for sentence in sentences_after_stopwords_puncts_lower:
        new_sentence = list()
        words_from_sentence = token_space.tokenize(sentence)
        for word in words_from_sentence:
            if word not in stop_words_no_accent:
                new_sentence.append(word)
        sentences_after_stopwords_puncts_lower_stopwords.append(" ".join(new_sentence))

    sentences_after_stopwords_puncts_lower_stopwords_number = list()

    for sentence in sentences_after_stopwords_puncts_lower_stopwords:
        new_sentence = list()
        words_from_sentence = token_space.tokenize(sentence)
        for word in words_from_sentence:
            if not word.isnumeric():
                new_sentence.append(word)
            else:
                new_sentence.append("0")
        sentences_after_stopwords_puncts_lower_stopwords_number.append(" ".join(new_sentence))

    return sentences_after_stopwords_puncts_lower_stopwords_number

In [ ]:
transformSentence(["i love this movie!!"])